In [31]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from fiona.crs import from_epsg
import geopandas as gpd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [32]:
df = pd.read_csv('vehicle_paths_pnas.csv')

In [33]:
gdf_point = gpd.GeoDataFrame(df)

In [34]:
gdf_point['Longitude'] = gdf_point['Longitude'].astype(float)
gdf_point['Latitude'] = gdf_point['Latitude'].astype(float)

In [35]:
gdf_point['lonlat'] = list(zip(gdf_point.Longitude,gdf_point.Latitude ))

In [36]:
gdf_point['geometry'] = gdf_point[['lonlat']].dropna().applymap(lambda x:Point(x))

In [37]:
gdf_point.crs = from_epsg(4326)

In [38]:
gdf_ct = gpd.GeoDataFrame.from_file('man.geojson')

In [39]:
all_ct_ids  = gdf_ct.ctlabel.unique().tolist()

In [40]:
gdf_ct.to_crs(epsg=2263, inplace=True)

In [41]:
gdf_point.to_crs(epsg=2263, inplace=True)

In [42]:
sj_output2 = gpd.sjoin(gdf_ct,gdf_point,how='inner')

In [43]:
sj_output2.ctlabel.unique()

array(['98', '100', '102', '104', '113', '114.02', '130', '140', '148.01',
       '184', '190', '206', '249', '261', '263', '267', '269', '277',
       '125', '119', '225', '293', '30.01', '56', '16', '120', '128',
       '121', '150.01', '106.01', '226', '55.01', '208', '134', '92',
       '114.01', '131', '65', '198', '200', '149', '227', '174.02', '215',
       '88', '90', '15.01', '86.02', '14.02', '70', '12', '224', '72',
       '74', '115', '106.02', '110', '116', '196', '82', '86.03', '124',
       '132', '255', '138', '84', '142', '136', '147', '151', '229', '233',
       '220', '31', '164', '158.02', '180', '182', '22.01', '22.02',
       '209.01', '58', '89', '93', '111', '223.01', '193', '75', '234',
       '214', '83', '307', '18', '41', '108', '112.03', '94', '96',
       '112.01', '112.02', '109', '291', '283', '285', '160.02', '186',
       '122', '183', '150.02', '160.01', '172', '79', '169', '159',
       '174.01', '170', '166', '245', '247', '165', '21', '101', '156.0

In [44]:
street_df = pd.DataFrame(sj_output2[['ctlabel','Num_Passengers']])

In [45]:
street_df.reset_index(inplace=True)

In [46]:
street_df.head()

,index,ctlabel,Num_Passengers
0,0,98,3
1,0,98,15
2,0,98,8
3,0,98,21
4,0,98,10


In [47]:
street_df.drop(['index'],axis=1,inplace=True)

In [48]:
street_df.rename(columns = {'Num_Passengers' : 'Avg_Passengers'},inplace = True)

In [49]:
street_df = street_df.groupby(['ctlabel']).mean()

In [50]:
street_df['Avg_Passengers'] = street_df['Avg_Passengers'].astype(int)

In [51]:
street_df.head()

,Avg_Passengers
ctlabel,
10.01,8
10.02,23
100,32
101,31
102,31


In [52]:
overcrowded_list_ids = street_df.index.tolist()

In [53]:
excl_st_ids = list(set(all_ct_ids).difference(set(overcrowded_list_ids)))

In [54]:
zero_pass_df = pd.DataFrame(excl_st_ids,columns=['ctlabel'])

In [55]:
zero_pass_df['Avg_Passengers'] = 0

In [56]:
zero_pass_df.set_index('ctlabel',inplace=True)

In [57]:
avg_pass = pd.concat([street_df,zero_pass_df],axis=0)

In [58]:
avg_pass.reset_index(inplace=True)

In [60]:
avg_pass.shape

(288, 2)

In [61]:
avg_pass.to_json('avg_pass_ct.json')

In [ ]:
sj_output2.set_index('ctlabel',inplace=True)

In [ ]:
merge_gdf = sj_output2.merge(street_df,left_index=True,right_index=True)

In [ ]:
merge_gdf.index.unique()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
gdf_ct.plot(ax=ax,color='grey')


In [ ]:
merge_gdf.plot(ax=ax,column='Avg_Passengers',cmap='viridis',legend=True)

In [ ]:
street_df.Num_Passengers.unique()

In [ ]:
street_df = street_df[street_df['Num_Passengers'] > -1]

In [ ]:
street_df.Num_Passengers.unique()

In [ ]:
street_df.to_csv('street_df.csv')

In [ ]:
street_df.head(20)

In [ ]:
street_df.reset_index(inplace=True)

In [ ]:
street_df.head()

In [ ]:
concat_street_df = street_df.groupby('id').apply(lambda x: (x['Num_Passengers'].values).tolist())

In [ ]:
concat_street_df = pd.DataFrame(concat_street_df)

In [ ]:
concat_street_df.rename(columns = {0 : 'num_pass'},inplace=True)

In [ ]:
concat_street_df.to_json('num_pass_list.json')